## Application des transformations definies dans "trasnfromers.py"

In [10]:
from transformers import *

In [11]:
import pandas as pd
from abc import ABC, abstractmethod
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pathlib import Path
from sklearn.model_selection import train_test_split

import pickle


In [12]:
columns_to_drop = [
        "piezo_station_department_name",
        "piezo_station_update_date",
        "piezo_station_commune_code_insee",
        "piezo_station_pe_label",
        "piezo_station_bdlisa_codes",
        "piezo_station_bss_code",
        "piezo_station_bss_id", 
        "piezo_bss_code",
        "piezo_measurement_date",
        "piezo_producer_name",
        "piezo_measure_nature_code",
        "meteo_name",
        "meteo_id", 
        "meteo_latitude",
        "meteo_longitude",
        "hydro_station_code",
        "hydro_method_code", 
        "hydro_method_label", 
        "hydro_qualification_label", 
        "prelev_structure_code_0",
        "prelev_structure_code_2",
        "prelev_structure_code_0",
        "prelev_commune_code_insee_0",
        "piezo_station_department_code",
        
        "meteo_DRR", 
        "meteo_temperature_min_ground", 
        "meteo_temperature_min_50cm", 
        "meteo_pressure_avg",
        "meteo_pression_maxi", 
        "meteo_wind_speed_avg_2m", 
        "meteo_wind_max_2m", 
        "meteo_wind_direction_max_inst_2m", 
        "meteo_time_wind_max_2m", 
        "meteo_wetting_duration", 
        "meteo_sunshine_duration", 
        "meteo_radiation", 
        "meteo_radiation_direct", 
        "meteo_sunshine_%", 
        "meteo_radiation_IR", 
        "meteo_radiation_UV_max", 
        "meteo_cloudiness", 
        "meteo_cloudiness_height", 
        "meteo_if_snow", 
        "meteo_if_fog", 
        "meteo_if_thunderstorm", 
        "meteo_if_sleet", 
        "meteo_if_hail", 
        "meteo_if_dew", 
        "meteo_if_black_ice", 
        "meteo_if_snow_ground", 
        "meteo_if_frost", 
        "meteo_if_smoke", 
        "meteo_if_mist",
        "meteo_if_lightning", 
        "meteo_evapotranspiration_Monteith", 
        "meteo_radiation_UV", 
        "meteo_snow_height", 
        "meteo_snow_thickness_max", 
        "meteo_snow_thickness_6h"]

ajouts_drop_yael = ["meteo_altitude","meteo_temperature_min","meteo_temperature_max"]


columns_to_drop+=ajouts_drop_yael

In [13]:
# Specification des colonnes  a garder 

##ANCIENNE VERSION
#all_columns = pd.read_csv(path_src_dataset, nrows=0).columns.tolist()
#columns_to_keep = [col for col in all_columns if col not in columns_to_drop] 


##NOUVELLE VERSION 
#Je précise ci dessous les colonnes nécessaires pour chacun des transformers, et en commentaire l'ordre dans lequel ce transformer sera utilisé
#Date
col_yass = ['meteo_date'] + ['prelev_volume_0', 'prelev_volume_1',
                             'prelev_volume_2', 'prelev_other_volume_sum','piezo_station_commune_name']  # ORDRE = 4 (en dernier car il supprime les dates )
#Altitude
altitude_flo = ["piezo_station_altitude", "meteo_altitude"] # ORDRE 1
prelev_flo = ["prelev_volume_0", "prelev_usage_label_0", "prelev_volume_obtention_mode_label_0", "prelev_volume_1", "prelev_usage_label_1", "prelev_volume_obtention_mode_label_1", "prelev_volume_2", "prelev_usage_label_2", "prelev_volume_obtention_mode_label_2"]
col_flo = altitude_flo + prelev_flo
#Insee & rain "CleanFeatures"
cols_yael_input = ['insee_%_agri', 'meteo_rain_height', 'insee_pop_commune', 'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
cols_yael_need = ["piezo_station_department_code", "meteo_date"]

#Temperature
cols_lucien_need = ['piezo_station_department_code', 'piezo_measurement_date']
cols_lucien_input = ['meteo_temperature_avg','meteo_temperature_min','meteo__pressure_saturation_avg','meteo_temperature_max']
#Lat Long
cols_mat = ["distance_piezo_meteo",'piezo_station_longitude','piezo_station_latitude','meteo_latitude','meteo_longitude', "meteo_temperature_avg", "meteo_temperature_avg_threshold", "meteo_temperature_min", "meteo_temperature_max", "meteo_temperature_min_ground", "hydro_observation_result_elab", "hydro_status_code", "hydro_qualification_code", "hydro_hydro_quantity_elab"]


#Clean pizo
pizo_cols = ['piezo_station_investigation_depth', 'piezo_obtention_mode', 'piezo_status', 'piezo_qualification', 'piezo_measure_nature_code','piezo_station_department_code']
 
#target
target = "piezo_groundwater_level_category"

columns_to_keep = col_yass + cols_yael_input + cols_yael_need + col_flo + cols_lucien_need + cols_lucien_input + cols_mat + [target] + pizo_cols

columns_to_keep

['meteo_date',
 'prelev_volume_0',
 'prelev_volume_1',
 'prelev_volume_2',
 'prelev_other_volume_sum',
 'insee_%_agri',
 'meteo_rain_height',
 'insee_pop_commune',
 'insee_med_living_level',
 'insee_%_ind',
 'insee_%_const',
 'piezo_station_department_code',
 'meteo_date',
 'piezo_station_altitude',
 'meteo_altitude',
 'prelev_volume_0',
 'prelev_usage_label_0',
 'prelev_volume_obtention_mode_label_0',
 'prelev_volume_1',
 'prelev_usage_label_1',
 'prelev_volume_obtention_mode_label_1',
 'prelev_volume_2',
 'prelev_usage_label_2',
 'prelev_volume_obtention_mode_label_2',
 'piezo_station_department_code',
 'piezo_measurement_date',
 'meteo_temperature_avg',
 'meteo_temperature_min',
 'meteo__pressure_saturation_avg',
 'meteo_temperature_max',
 'distance_piezo_meteo',
 'piezo_station_longitude',
 'piezo_station_latitude',
 'meteo_latitude',
 'meteo_longitude',
 'meteo_temperature_avg',
 'meteo_temperature_avg_threshold',
 'meteo_temperature_min',
 'meteo_temperature_max',
 'meteo_tempera

In [19]:
path_src_dataset = Path("./data/src/X_train_Hi5.csv")


# Out folders
out_folder_dataset = Path("./data/cleaned")
out_folder_dataset.mkdir(parents=True, exist_ok=True) # Create the folder if it doesn't exist
out_folder_config = Path("./data/processed/pipelines")
out_folder_config.mkdir(parents=True, exist_ok=True)

# Load the CSV file with only the relevant columns
df = pd.read_csv(path_src_dataset, usecols=columns_to_keep) #  SI on veut charger moins de lignes : ajouter --> ,nrows=10e4)
df = df.drop_duplicates()



X = df.drop(columns=target)

#Mapping du target 
mapping = {'Very Low': 0, 'Low': 1, 'Average': 2, 'High': 3, 'Very High': 4}
y = df[target].map(mapping)

#Test-val split 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Apply the transformers selected
processing_pipeline = Pipeline(steps=[
("DropNaRate", DropNaRate(0.7)),
("Prelevol",PrelevVol()),
("Prelevement", Prelev(columns=col_flo,usage_label_max_categories=4,mode_label_max_categories=4,scale=1)),
("CleanFeatures", CleanFeatures(cols_yael_input)),
("Altitude", AltitudeTrans(columns=["piezo_station_altitude", "meteo_altitude"])),
('LatLong',CleanLatLon()),
('CleanTemp',CleanTemp()),
('Temp',TemperaturePressionTrans(columns=cols_lucien_input)),
('CleanHydro', CleanHydro()),
('CleanPizo',  CleanPizo(pizo_cols)),
('Dates',DateTransformer()),
('DropCols',DropCols(columns_to_drop))

# ... Add others transformations
])

/var/folders/1h/b1s2w7bs0rd5fvjqfq1pbky80000gn/T/ipykernel_25324/3382408009.py:11: DtypeWarning: Columns (110,111,117,118,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_src_dataset, usecols=columns_to_keep) #  SI on veut charger moins de lignes : ajouter --> ,nrows=10e4)


In [17]:
print("Pipeline ongoing...")
processed_X_train = processing_pipeline.fit_transform(X_train)
processed_X_val = processing_pipeline.transform(X_val)

Pipeline ongoing...
>> (Info) Droped columns : []
>> (Info - Prelev) 'prelev_usage_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_0' has been one-hot-encoded in 4 features
>> (Info) Recuperations des moyennes des données INSEE par department
>> (Info) Infos medianes Insee recupérees
>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (Info) Calculating means for numerical features and preparing for one-hot encoding.
>> (Info) Fitting completed: Means, modes, and one-hot encoders prepared.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding applied to piezo_qualification with missing values filled.
>> (Info) One-h

In [7]:
processed_X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79614 entries, 0 to 79613
Data columns (total 28 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   piezo_station_altitude                                   79614 non-null  float64
 1   piezo_station_longitude                                  79614 non-null  float64
 2   piezo_station_latitude                                   79614 non-null  float64
 3   meteo_date                                               79614 non-null  float64
 4   meteo_rain_height                                        79614 non-null  float64
 5   meteo_temperature_avg                                    79614 non-null  float64
 6   meteo_temperature_avg_threshold                          67380 non-null  float64
 7   meteo__pressure_saturation_avg                           79614 non-null  float64
 8   distance_piezo_meteo      

In [18]:
print("Pipeline ongoing...")
processed_X_train = processing_pipeline.fit_transform(X_train)
processed_X_val = processing_pipeline.transform(X_val)
processed_X_train.describe()

# Save the processed data to CSV
# processed_X_train.to_csv(out_folder_dataset / "X_train.csv", index=False)
# processed_X_val.to_csv(out_folder_dataset / "X_val.csv", index=False)
# y_train.to_csv(out_folder_dataset / "y_train.csv", index=False)
# y_val.to_csv(out_folder_dataset / "y_val.csv", index=False)

# print("Data converted to csv")

Pipeline ongoing...
>> (Info) Droped columns : []
>> (Info - Prelev) 'prelev_usage_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_0' has been one-hot-encoded in 4 features
>> (Info) Recuperations des moyennes des données INSEE par department
>> (Info) Infos medianes Insee recupérees
>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (Info) Calculating means for numerical features and preparing for one-hot encoding.
>> (Info) Fitting completed: Means, modes, and one-hot encoders prepared.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding applied to piezo_qualification with missing values filled.
>> (Info) One-h

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,meteo_date,meteo_rain_height,meteo_temperature_avg,meteo_temperature_avg_threshold,meteo__pressure_saturation_avg,distance_piezo_meteo,...,piezo_status_Donnée interprétée,piezo_qualification_Correcte,piezo_qualification_Incertaine,piezo_qualification_Incorrecte,piezo_qualification_Non qualifié,piezo_measure_nature_code_0,piezo_measure_nature_code_D,piezo_measure_nature_code_I,piezo_measure_nature_code_N,piezo_measure_nature_code_S
count,2.263711e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.071551e+06,2.264252e+06,2.264252e+06,...,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06,2.264252e+06
mean,8.517859e+01,1.505220e+02,2.214996e+00,4.641199e+01,1.073117e-01,2.077347e+00,1.177645e+01,1.210186e+01,1.086808e+01,9.500435e-01,...,9.154016e-03,9.606161e-01,3.107870e-03,3.330901e-03,3.294510e-02,5.425399e-01,6.907358e-03,4.280663e-02,4.068858e-01,8.603283e-04
std,2.369374e+02,1.808122e+02,2.722042e+00,2.174460e+00,6.821896e-01,5.169685e+00,6.217075e+00,6.214249e+00,3.895757e+00,2.178551e-01,...,9.523773e-02,1.945065e-01,5.566159e-02,5.761777e-02,1.784929e-01,4.981872e-01,8.282300e-02,2.024210e-01,4.912533e-01,2.931874e-02
min,0.000000e+00,7.200000e-01,-4.657129e+00,4.139973e+01,-9.999792e-01,0.000000e+00,-1.500000e+01,-1.495000e+01,2.000000e-01,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.700000e+01,4.158000e+01,-1.086572e-01,4.449116e+01,-5.476112e-01,0.000000e+00,7.400000e+00,7.650000e+00,7.900000e+00,1.000000e+00,...,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.000000e+01,1.080000e+02,1.899878e+00,4.638521e+01,1.933765e-01,2.000000e-01,1.144306e+01,1.180000e+01,1.040000e+01,1.000000e+00,...,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.000000e+01,1.859500e+02,4.197586e+00,4.820457e+01,7.724560e-01,1.600000e+00,1.630000e+01,1.665000e+01,1.350000e+01,1.000000e+00,...,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
max,5.233400e+03,2.150000e+03,9.519890e+00,5.097109e+01,1.000000e+00,2.102000e+02,3.150000e+01,3.225000e+01,2.830000e+01,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [13]:
processed_X_train.columns

Index(['piezo_station_investigation_depth', 'piezo_station_altitude',
       'piezo_station_longitude', 'piezo_station_latitude', 'meteo_date',
       'meteo_rain_height', 'meteo_temperature_avg',
       'meteo__pressure_saturation_avg', 'distance_piezo_meteo',
       'hydro_observation_result_elab', 'prelev_volume_0',
       'prelev_other_volume_sum', 'insee_%_agri', 'insee_pop_commune',
       'insee_med_living_level', 'insee_%_ind', 'insee_%_const',
       'prelev_usage_label_0_CANAUX', 'prelev_usage_label_0_EAU POTABLE',
       'prelev_usage_label_0_EAU TURBINEE (barrage)',
       'prelev_usage_label_0_infrequent_sklearn',
       'prelev_volume_obtention_mode_label_0_Mesure directe',
       'prelev_volume_obtention_mode_label_0_Mesure indirecte',
       'prelev_volume_obtention_mode_label_0_Volume mesuré',
       'prelev_volume_obtention_mode_label_0_infrequent_sklearn',
       'hydro_observation_log',
       'piezo_obtention_mode_Mode d'obtention inconnu',
       'piezo_obtention_

In [10]:
a

['insee_med_living_level',
 'hydro_observation_result_elab',
 'piezo_station_longitude',
 'insee_%_ind',
 'meteo_temperature_avg_threshold',
 'hydro_hydro_quantity_elab_infrequent_sklearn',
 'meteo_temperature_avg',
 'insee_pop_commune',
 'insee_%_const',
 'hydro_qualification_code',
 'piezo_station_latitude',
 'hydro_observation_log',
 'insee_%_agri',
 'meteo_date',
 'meteo__pressure_saturation_avg',
 'piezo_station_altitude',
 'prelev_volume_0',
 'hydro_status_code',
 'meteo_rain_height']

# Quick KNN

In [ ]:
#quick KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score


# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=25)  # Default is 5 neighbors
knn.fit(processed_X_train, y_train)

# Make predictions
y_val_pred = knn.predict(processed_X_val)

# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))


processed_X_train.head()

In [20]:
#quick KNN
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

global_pipeline = Pipeline([
    ('processing', processing_pipeline),
    ('StandardScaler', PartialStandardScaler(columns='all')),
    ('estimator', RandomForestClassifier())
])

In [21]:
global_pipeline.fit(X_train, y_train)

>> (Info) Droped columns : []
>> (Info - Prelev) 'prelev_usage_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_0' has been one-hot-encoded in 4 features
>> (Info) Recuperations des moyennes des données INSEE par department
>> (Info) Infos medianes Insee recupérees
>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (Info) Calculating means for numerical features and preparing for one-hot encoding.
>> (Info) Fitting completed: Means, modes, and one-hot encoders prepared.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding applied to piezo_qualification with missing values filled.
>> (Info) One-hot encoding applied 

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [22]:
processed_X_train.isna().sum()

piezo_station_investigation_depth                             541
piezo_station_altitude                                          0
piezo_station_longitude                                         0
piezo_station_latitude                                          0
meteo_date                                                      0
meteo_rain_height                                               0
meteo_temperature_avg                                           0
meteo_temperature_avg_threshold                            192701
meteo__pressure_saturation_avg                                  0
distance_piezo_meteo                                            0
hydro_observation_result_elab                                   0
hydro_status_code                                               0
hydro_qualification_code                                        0
prelev_volume_0                                            441694
prelev_other_volume_sum                                    303129
insee_%_ag

In [ ]:
y_val_pred = global_pipeline.predict(X_val)

# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))

>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding applied to piezo_qualification with missing values filled.
>> (Info) One-hot encoding applied to piezo_measure_nature_code with missing values filled.
>> (Info) Data transformation completed.
>> (INFO - DropCols) columns ['piezo_station_department_name', 'piezo_station_update_date', 'piezo_station_commune_code_insee', 'piezo_station_pe_label', 'piezo_station_bdlisa_codes', 'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code', 'piezo_measurement_date', 'piezo_producer_name', 'piezo_measure_nature_code', 'meteo_name', 'meteo_id', 'meteo_latitude', 'meteo_lo

In [ ]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))

Accuracy: 0.58225

Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.56      0.59      1934
           1       0.58      0.50      0.54      3222
           2       0.56      0.56      0.56      4630
           3       0.56      0.60      0.58      5415
           4       0.62      0.65      0.63      4799

    accuracy                           0.58     20000
   macro avg       0.59      0.57      0.58     20000
weighted avg       0.58      0.58      0.58     20000



In [ ]:
rf = global_pipeline.named_steps['estimator']

rf.feature_importances_

array([0.05986686, 0.05502813, 0.0547437 , 0.14608801, 0.0411407 ,
       0.0774364 , 0.04493748, 0.00192649, 0.12639811, 0.07688095,
       0.03314264, 0.03700347, 0.03694276, 0.02944164, 0.03252412,
       0.00327626, 0.00201415, 0.00176134, 0.00373462, 0.00302471,
       0.00181662, 0.00179391, 0.00249644, 0.1265805 ])

### Save Pipeline

In [ ]:
save = True
if save:
        
    pipeline_name = "pipeline_randomforest_1st"

    # Writing to sample.json
    with open(out_folder_config / Path(pipeline_name + ".pkl"), "wb") as file:
        pickle.dump(global_pipeline, file)